<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 4B: Exploring SAR Data and SAR Time Series Analysis using Jupyter Notebooks </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It introduces you to the analysis of deep multi-temporal SAR image data stacks in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>In this chapter we introduce the following data analysis concepts:</b>

- How to load your own SAR data into Jupyter Notebooks and create a time series stack 
- How to apply calibration constants to covert initial digital number (DN) data into calibrated radar cross section information.
- How to subset images and create a time series of your subset data.
- How to explore the time-series information in SAR data stacks for environmental analysis.


This Notebook is an addon to Lab 4 and doesn't include any homework assignments. Free free to play around with your own data and contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [1]:
# Scientific Libraries and hyp3 API
import pandas as pd # for DatetimeIndex
import gdal # for gdalbuildvr, gdalinfo, gdalmerge, gdalwarp, gdal_translate, Open
import numpy as np # for copy, isnan, log10, ma.masked_where, max, mean, min, percentile, power, unique, var, where 
import matplotlib.pylab as plb # for figure, grid, rcParams, savefig
from asf_hyp3 import API # for get_products, get_subscriptions, login
import scipy # for signal.savgol_filter()
import scipy.signal

from IPython.display import HTML

import matplotlib.pyplot as plt
#import matplotlib.animation
from matplotlib import animation
from matplotlib import rc
import numpy as np


# General Purpose Libraries
import os # for chdir, getcwd, path.exists
import glob # for glob
import re # for match
from getpass import getpass # used to input URS creds and add to .netrc

<font face="Calibri" size="3"><b>set up matplotlib plotting</b> inside the notebook:</font>

In [2]:
%matplotlib inline 

<font face="Calibri" size="3"> <b>Write a helper function</b> to check that a given file or path exists. This provides simple error handling for some of our os library function calls:</font>

In [3]:
def pathExists(path):
    if os.path.exists(path):
        return True
    else:
        print(f"Invalid Path: {path}")
        return False

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Your Own Data Stack Into the Notebook </b> </font> 

<font size="3"> This lab assumes that you've created your own data stack over your personal area of interest using the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility's</a> value-added product system <a href="http://hyp3.asf.alaska.edu/" target="_blank">HyP3</a>. HyP3 is an environment that is used by ASF to prototype value added products and provide them to users to collect feedback. 

This lab expects <a href="https://media.asf.alaska.edu/uploads/RTC/rtc_atbd_v1.2_final.pdf" target="_blank">Radiometric Terrain Corrected</a> (RTC) image products as input, so be sure to select an RTC process when creating the subscription for your input data within HyP. Prefer a **unique orbit geometry** (ascending or descending) to keep geometric differences between images low. 

We will retrieve HyP3 data via the HyP3 API. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font> 
</font>

<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. <b>The following field allows you to store your NASA Earth Data <font color='rgba(200,0,0,0.2)'>username and password</font> in this notebook for later use in data downloading:</b> </font>

In [4]:
NEDusr = 'aflewandowski'
NEDpass = getpass()
filename='/home/jovyan/.netrc'
with open(filename, 'w+') as f:
    f.write(f"machine urs.earthdata.nasa.gov login {NEDusr} password {NEDpass}\n")

········


<hr>
<font face="Calibri" size="3"> Before we download anything, let's <b>first create a working directory for this analysis and change into it:</b> </font>

In [5]:
path = '/home/jovyan/notebooks/ASF/GEOS_657_Labs'
if pathExists(path):
    os.chdir(path)
    !mkdir -p lab_4B_data
    path = path + '/lab_4B_data'
    if pathExists(path):
        os.chdir(path)
print(f"current directory: {os.getcwd()}")

current directory: /home/jovyan/notebooks/ASF/GEOS_657_Labs/lab_4B_data


<hr>
<font face="Calibri" size="3"> With your username/password now in place, you can now <b>log into the HyP3 API</b> (Documentation <a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">here</a>):</font>

In [6]:
api = API(NEDusr)
api.login(NEDpass)


 login successful!


<font face="Calibri" size="3">Now,<b> Query your existing subscriptions.</b> </font>

In [7]:
subs = api.get_subscriptions(enabled=True)
for sub in subs:
    print(f"\nSubscription id: {sub['id']} {sub['name']}")


Subscription id: 1638 MedicineLake_InSAR_GAMMA

Subscription id: 1643 MedicineLake_RTC_S1TBX

Subscription id: 1641 MedicineLake_RTC_GAMMA


<hr>
<font face="Calibri" size="3"> Look through your existing subscriptions and <b>select the subscription ID</b> you would like to work with. <b>Enter the selected ID in the code cell below</b> (replace "1478" with the ID of your choice). </font>

In [8]:
# Look at products in subscription and print out download urls
subscription_id = 1643
products = api.get_products(sub_id=subscription_id)

print(f"\nURLs of products: \n")
for p in products:
    print(f"{p['url']}\n")


URLs of products: 

https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20180506T032013_20180506T032038_010795_013BB3_DA74-PREDORB-30m-rtc-s1tbx.zip

https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20190220T030359_20190220T030424_015024_01C125_8554-PREDORB-30m-rtc-s1tbx.zip

https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181114T032021_20181114T032046_013595_0192AE_64B6-PREDORB-30m-rtc-s1tbx.zip

https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181128T030402_20181128T030427_013799_01992D_EF8B-PREDORB-30m-rtc-s1tbx.zip

https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20190115T030400_20190115T030425_014499_01B00C_36FF-PREDORB-30m-rtc-s1tbx.zip

https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20180820T161144_20180820T161209_023332_0289AD_33F5-PREDORB-30m-rtc-s1tbx.zip

https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20180822T032019_20180822T032044_012370_016CDD_C9AD-PREDORB-30m-rtc-s1t

<hr>
<font face="Calibri" size="3"> Now you are ready to <b>download</b> all files associated with your subscription. </font>

In [9]:
# Download products from subscription.
!mkdir -p granules
if pathExists(f"{path}/granules"):
    for p in products:
        url = p['url']
        _match = re.match(r'https://hyp3-download.asf.alaska.edu/asf/data/(.*).zip', url)
        granule = _match.group(1)

        filename = f"granules/{granule}"
        # Guess we need to download and unzip
        if not pathExists(filename):
            print(f"\n{granule} is not present.\nDownloading from {url}")
            !wget -O granules/{granule}.zip "{url}"
            !unzip -d granules granules/{granule}.zip
            !rm granules/{granule}.zip


Invalid Path: granules/S1B_IW_GRDH_1SDV_20180506T032013_20180506T032038_010795_013BB3_DA74-PREDORB-30m-rtc-s1tbx

S1B_IW_GRDH_1SDV_20180506T032013_20180506T032038_010795_013BB3_DA74-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20180506T032013_20180506T032038_010795_013BB3_DA74-PREDORB-30m-rtc-s1tbx.zip
--2019-05-28 23:59:13--  https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20180506T032013_20180506T032038_010795_013BB3_DA74-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.232.176.77, 34.200.77.222
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.232.176.77|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1B_IW_GRDH_1SD

Invalid Path: granules/S1B_IW_GRDH_1SDV_20190220T030359_20190220T030424_015024_01C125_8554-PREDORB-30m-rtc-s1tbx

S1B_IW_GRDH_1SDV_20190220T030359_20190220T030424_015024_01C125_8554-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20190220T030359_20190220T030424_015024_01C125_8554-PREDORB-30m-rtc-s1tbx.zip
--2019-05-28 23:59:24--  https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20190220T030359_20190220T030424_015024_01C125_8554-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.200.77.222, 34.232.176.77
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.200.77.222|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1B_IW_GRDH_1SD

Invalid Path: granules/S1B_IW_GRDH_1SDV_20181114T032021_20181114T032046_013595_0192AE_64B6-PREDORB-30m-rtc-s1tbx

S1B_IW_GRDH_1SDV_20181114T032021_20181114T032046_013595_0192AE_64B6-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181114T032021_20181114T032046_013595_0192AE_64B6-PREDORB-30m-rtc-s1tbx.zip
--2019-05-28 23:59:32--  https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181114T032021_20181114T032046_013595_0192AE_64B6-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.232.176.77, 34.200.77.222
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.232.176.77|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1B_IW_GRDH_1SD

Invalid Path: granules/S1B_IW_GRDH_1SDV_20181128T030402_20181128T030427_013799_01992D_EF8B-PREDORB-30m-rtc-s1tbx

S1B_IW_GRDH_1SDV_20181128T030402_20181128T030427_013799_01992D_EF8B-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181128T030402_20181128T030427_013799_01992D_EF8B-PREDORB-30m-rtc-s1tbx.zip
--2019-05-28 23:59:40--  https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181128T030402_20181128T030427_013799_01992D_EF8B-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.200.77.222, 34.232.176.77
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.200.77.222|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1B_IW_GRDH_1SD

Invalid Path: granules/S1B_IW_GRDH_1SDV_20190115T030400_20190115T030425_014499_01B00C_36FF-PREDORB-30m-rtc-s1tbx

S1B_IW_GRDH_1SDV_20190115T030400_20190115T030425_014499_01B00C_36FF-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20190115T030400_20190115T030425_014499_01B00C_36FF-PREDORB-30m-rtc-s1tbx.zip
--2019-05-28 23:59:49--  https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20190115T030400_20190115T030425_014499_01B00C_36FF-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.232.176.77, 34.200.77.222
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.232.176.77|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1B_IW_GRDH_1SD

...

Invalid Path: granules/S1B_IW_GRDH_1SDV_20181220T032020_20181220T032045_014120_01A3AB_AF40-PREDORB-30m-rtc-s1tbx

S1B_IW_GRDH_1SDV_20181220T032020_20181220T032045_014120_01A3AB_AF40-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181220T032020_20181220T032045_014120_01A3AB_AF40-PREDORB-30m-rtc-s1tbx.zip
--2019-05-29 00:11:00--  https://hyp3-download.asf.alaska.edu/asf/data/S1B_IW_GRDH_1SDV_20181220T032020_20181220T032045_014120_01A3AB_AF40-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.232.176.77, 34.200.77.222
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.232.176.77|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1B_IW_GRDH_1SD

Invalid Path: granules/S1A_IW_GRDH_1SDV_20180528T161139_20180528T161213_022107_0263D8_0C69-PREDORB-30m-rtc-s1tbx

S1A_IW_GRDH_1SDV_20180528T161139_20180528T161213_022107_0263D8_0C69-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20180528T161139_20180528T161213_022107_0263D8_0C69-PREDORB-30m-rtc-s1tbx.zip
--2019-05-29 00:11:14--  https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20180528T161139_20180528T161213_022107_0263D8_0C69-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.232.176.77, 34.200.77.222
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.232.176.77|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1A_IW_GRDH_1SD

Invalid Path: granules/S1A_IW_GRDH_1SDV_20180703T161141_20180703T161206_022632_0273BB_1A39-PREDORB-30m-rtc-s1tbx

S1A_IW_GRDH_1SDV_20180703T161141_20180703T161206_022632_0273BB_1A39-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20180703T161141_20180703T161206_022632_0273BB_1A39-PREDORB-30m-rtc-s1tbx.zip
--2019-05-29 00:11:27--  https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20180703T161141_20180703T161206_022632_0273BB_1A39-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.200.77.222, 34.232.176.77
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.200.77.222|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1A_IW_GRDH_1SD

Invalid Path: granules/S1A_IW_GRDH_1SDV_20190111T161144_20190111T161209_025432_02D14C_A2A8-PREDORB-30m-rtc-s1tbx

S1A_IW_GRDH_1SDV_20190111T161144_20190111T161209_025432_02D14C_A2A8-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20190111T161144_20190111T161209_025432_02D14C_A2A8-PREDORB-30m-rtc-s1tbx.zip
--2019-05-29 00:11:36--  https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20190111T161144_20190111T161209_025432_02D14C_A2A8-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.232.176.77, 34.200.77.222
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.232.176.77|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1A_IW_GRDH_1SD

Invalid Path: granules/S1A_IW_GRDH_1SDV_20181230T161144_20181230T161209_025257_02CB01_FE5E-PREDORB-30m-rtc-s1tbx

S1A_IW_GRDH_1SDV_20181230T161144_20181230T161209_025257_02CB01_FE5E-PREDORB-30m-rtc-s1tbx is not present. Downloading from https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20181230T161144_20181230T161209_025257_02CB01_FE5E-PREDORB-30m-rtc-s1tbx.zip
--2019-05-29 00:11:49--  https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_GRDH_1SDV_20181230T161144_20181230T161209_025257_02CB01_FE5E-PREDORB-30m-rtc-s1tbx.zip
Resolving hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)... 34.200.77.222, 34.232.176.77
Connecting to hyp3-download.asf.alaska.edu (hyp3-download.asf.alaska.edu)|34.200.77.222|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=hpE7B3RwdHdZNApa8DhnBQ&response_type=code&redirect_uri=https://hyp3-download.asf.alaska.edu/login&state=asf%2Fdata%2FS1A_IW_GRDH_1SD

<hr>
<font face="Calibri" size="3"> Run the following code cell to visualize the image acquisition dates in your subscription. </font>

In [ ]:
# We need dates for
path = "granules"
if pathExists(path):
    paths = f"{path}/*/*_VV.tif" # filename format for RTC-GAMMA
    g = glob.glob(paths)
   # if not g:
   #     paths = f"{path}/*/*-VV.tif" # filename format for RTC-S1TBX
   #     g = glob.glob(paths)
    if not g:
        print("Note: This recipe requires RTC imagery.\nMake sure your Hyp subscription uses the RTC-GAMMA process.")
    else:
        !ls  $paths | sort -t_ -k5,5 | cut -c 27-34 > granule.dates
        !cat granule.dates

<hr>
<font face="Calibri" size="3"> You may notice duplicates in your acquisition dates. As HyP3 processes SAR data on a frame-by-frame basis, duplicates may occur if your area of interest is covered by two consecutive  image frames. In this case, two separate images are generated that need to be merged together before time series processing can commence. <b>The next code cell is identifying frames in need to merging and is mosaicking these frames together.</b> </font>

In [ ]:
# Grab the paths of the VV
tiff_paths = !ls $paths | sort -t_ -k5,5
print("Tiff paths:")
for p in tiff_paths:
    print(f"{p}\n")

<hr>
<font face="Calibri" size="3"> <b>Fixing multiple UTM Zones-related issues</b> should they exist in your data set. If multiple UTM zones are fond, the following code cells will identify the predominant UTM zone and reproject the rest into that zone. </font>

In [ ]:
!ls $paths | sort -t_ -k5,5 > granule.files
if pathExists('granule.dates'):
    with open('granule.dates', 'r') as d:
        dates = d.readlines()
if pathExists('granule.files'):
    with open('granule.files', 'r') as f:
        files = f.readlines()
output_file = ('utmzones.txt', 'w')
print('Checking UTM Zones in the data stack ...')
for  k in range(0, len(dates)):
    gdal_command = f"gdalinfo {tiff_paths[k]} | grep '^    AUTHORITY' | cut -d '\"' -f 2,4 | tr '\"' ':'"
    !{gdal_command} >> test
    if ((k+1)/len(dates)*100)%5 == 0:
        print("%i%% completed ..." % ((k+1)/len(dates)*100), end='\r', flush=True)
print('\nDone!')

In [ ]:
!cut -f 2 -d ':' test > utmzones
utmzones=[i.strip() for i in open('utmzones').readlines()]
utmzones2=[i.strip() for i in open('test').readlines()]

utmunique, counts = np.unique(utmzones, return_counts=True)
a = np.where(counts == np.max(counts))
maxutm = utmunique[a][0]
reproind = [i for i, j in enumerate(utmzones) if j != maxutm]
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproind)))
print('--------------------------------------------')
for k in reproind:
    temppath = files[k].strip()
    _, granule_name, tiff_name = temppath.split('/')
    cmd = f"gdalwarp -overwrite granules/{granule_name}/{tiff_name} granules/{granule_name}/r{tiff_name} -s_srs {utmzones2[k]} -t_srs EPSG:{maxutm}"
    #print(f"Calling the command: {cmd}")
    !{cmd}
    rm_command = f"rm {files[k].strip()}"
    #print(f"Calling the command: {rm_command}")
    !{rm_command}

<hr>
<font face="Calibri" size="3"> <b>Concatenate neighboring image frames</b> should your area be covered by more than one frame. </font>

In [ ]:
!ls $paths | sort -t_ -k5,5 > granule.files
#!cat granule.files
with open('granule.dates', 'r') as f:
    dates = f.readlines()
print(dates)
for  k in range(1, len(dates)):
    if dates[k] == dates[k-1]:
        print(k)
        temp = tiff_paths[k-1]
        _, granule_name, tiff_name = temp.split('/')
        gdal_command = f"gdal_merge.py -o granules/{granule_name}/new-{tiff_name} {tiff_paths[k]} {tiff_paths[k-1]}"
        print(f"Calling the command: {gdal_command}")
        !{gdal_command}
        rm_command = f"rm {tiff_paths[k]}"
        print(f"Calling the command: {rm_command}")
        !{rm_command}
        rm_command = f"rm {tiff_paths[k-1]}"
        print(f"Calling the command: {rm_command}")
        !{rm_command}
        

<hr>
<font face="Calibri" size="3"> Let's verify that all date duplicates were resolved: </font>

In [ ]:
# We need dates for
!ls $paths | sort -t_ -k5,5 | cut -c 27-34 > granule.dates
!cat granule.dates

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Create Subset and Stack Up Your Data </b> </font> 

<font size="3"> Now you are ready to work with your data. The next cells allow you to select an area of interest (AOI; via bounding-box corner coordinates) for your data analysis. Once selected, the AOI is being extracted and a data stack is formed.

<b>As a first step, we extract your AOI from the full frames:</b>
</font> 
</font>

In [ ]:
'''
# Example #1: Using Google Maps, get the rough bounding box for the subset
# These are the corner coordinates focused on the location of the 2018 Camp Fire
ulx = -121.7
lrx = -121.4
lry = 39.7
uly = 39.9
!echo {ulx} {lrx} {lry} {uly}
'''

In [ ]:
'''
# Example #2: Using Google Maps, get the rough bounding box for the subset
# Corner coordinates for the Huntsville Ag and Forest Time Series
ulx = -86.75
lrx = -86.3
lry = 34.6
uly = 34.88
!echo {ulx} {lrx} {lry} {uly}
'''

In [ ]:
# Using Google Maps, get the rough bounding box for the subset
# Enter your corner coordinates below
upper_left_x = -144.58
lower_right_x = -144.48
lower_right_y = 65.48
upper_left_y = 65.53
print(f"upper left x coord: {upper_left_x}\nupper left y coord: {upper_left_y}\nlower right x coord: {lower_right_x}\nlower right y coord: {lower_right_y}")

In [ ]:
# Grab the paths of the VV
tiff_paths = !ls $paths | sort  -t_ -k5,5

print("Tiff paths:")
for p in tiff_paths:
    print(f"{p}\n")

In [ ]:
# Cycle through and subset the tiffs in the products
!mkdir -p tiffs
if pathExists('tiffs'):
    for tiff_path in tiff_paths:
        _, granule_name, tiff_name = tiff_path.split('/')
        g1, g2, g3, date, g4, g5, g6 = tiff_name.split('_')
        # Using the GDAL subset service, get a small subset around the Butte
        #!wget -O {granule_name}_VV.tiff "https://services.asf.alaska.edu/geospatial/subset?ulx={ulx}&lrx={lrx}&lry={lry}&uly={uly}&product={granule_name}.zip/{granule_name}/{tiff_name}"

        # GDAL service is out of service. Pretend that it isn't when calling the following equivalent command
        gdal_command = f"gdal_translate -projwin {upper_left_x} {upper_left_y} {lower_right_x} {lower_right_y} -projwin_srs 'WGS84' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} tiffs/{date}_VV.tiff"
        print(f"Calling the command: {gdal_command}")
        !{gdal_command}


In [ ]:
# Get info on the last subsetted tiff
!gdalinfo tiffs/{date}_VV.tiff

<hr>
<font face="Calibri" size="3"> Now we stack up your data by creating a virtual raster table with links to all subset data files: </font>

In [ ]:
# Create the VRT for the downloaded subset geotiffs
# Grab all tiffs in the directory
!gdalbuildvrt -separate granule.vrt tiffs/*.tiff

In [ ]:
# We need dates for
!ls tiffs/*_VV.tiff | sort | cut -c 7-21 > granule.dates
!cat granule.dates

<hr>
<font face="Calibri">

<font size="5"> <b> 3. Now You Can Work With Your Data </b> </font> 

<font size="3"> Now you are ready to perform time series analysis on your data stack
</font> 
</font>

<br>
<font face="Calibri" size="4"> <b> 3.1 Define Data Directory and Path to VRT </b> </font> 

<font face="Calibri" size="3"> Just some path definitions. </font>

In [ ]:
# Set some paths
datadirectory = "/home/jovyan/notebooks/GEOS 657 Labs/lab_4B_data"
datefile = "granule.dates"
imagefile = "granule.vrt"

In [ ]:
# Get some indices for plotting
if pathExists(datefile):
    with open(datefile,'r') as f:
        dates=f.readlines()
        tindex=pd.DatetimeIndex(dates)

In [ ]:
# Bands and times
j=1
print(f"Bands and dates for {imagefile}")
for i in tindex:
    print("{:4d} {}".format(j, i.date()),end=' ')
    j+=1
    if j%5==1: print()

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.2 Open Your Data Stack and Visualize Some Layers </b> </font> 

<font face="Calibri" size="3"> We will open your VRT and visualize some layers using Matplotlib. </font>

In [ ]:
# Open virtual dataset
img=gdal.Open(imagefile)

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

In [ ]:
# Read in raster data for the first two bands
raster_1 = img.GetRasterBand(1).ReadAsArray()
where_are_NaNs = np.isnan(raster_1)
raster_1[where_are_NaNs] = 0

raster_3 = img.GetRasterBand(16).ReadAsArray()
where_are_NaNs = np.isnan(raster_3)
raster_3[where_are_NaNs] = 0


In [ ]:
# Plot some things
fig = plb.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot

# First plot: Image
bandnbr=1
ax1.imshow(raster_1,cmap='gray',vmin=0,vmax=0.2) #,vmin=2000,vmax=10000)
ax1.set_title('Image Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax2.hist(raster_1.flatten(),bins=200,range=(0,0.3))
ax2.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax2.set_title('Histogram Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))


# Third plot: Image
bandnbr=2
ax3.imshow(raster_3,cmap='gray',vmin=0,vmax=0.2) #,vmin=2000,vmax=10000)
ax3.set_title('Image Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

# Fourth plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax4.hist(raster_3.flatten(),bins=200,range=(0,0.3))
ax4.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax4.set_title('Histogram Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.3 Calibration and Data Conversion between dB and Power Scales </b> </font>

<font face="Calibri" size="3"> <font color='rgba(200,0,0,0.2)'> <b>Note, that if your data were generated by HyP3, this step is not necessary!</b> HyP3 performs the full data calibration and provides you with calibrated data in power scale. </font>
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 
<br> <br>
$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

<b>To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell</b>: </font> 

In [ ]:
# caldB=20*np.log10(rasterstack)-83

<font face="Calibri" size="3"> While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ </font>

In [ ]:
#calPwr=np.power(10.,caldB/10.)

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.4 Create a Time Series Animation </b> </font>

<font face="Calibri" size="3"> Now we are ready to <b>create a time series animation</b> from the calibrated SAR data. </font> 

In [ ]:
band = img.GetRasterBand(1)

In [ ]:
raster0 = band.ReadAsArray()
bandnbr=0 # Needed for updates
rasterstack=img.ReadAsArray()

In [ ]:
rs2 = np.ma.masked_where(rasterstack == 0, rasterstack)

In [ ]:
%%capture 
#import matplotlib.pyplot as plt
#import matplotlib.animation
#import numpy as np

fig=plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(rasterstack.flatten(),5)
vmax=np.percentile(rasterstack.flatten(),95)

r0dB=20*np.log10(raster0)-83

im = ax.imshow(raster0,cmap='gray',vmin=vmin,vmax=vmax)
ax.set_title("{}".format(tindex[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])

# Interval is given in milliseconds
ani = matplotlib.animation.FuncAnimation(fig, animate, 
                                         frames=rasterstack.shape[0],
                                        interval=400)

In [ ]:
rc('animation',embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
ani.save('animation.gif', writer='pillow', fps=2)

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.5 Plot the Time Series of Means Calculated Across the Subset </b> </font>

<font face="Calibri" size="3"> To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
2. compute means.
3. convert the resulting mean values into dB scale for visualization.
4. plot time series of means. </font> 

In [ ]:
# 2. Compute Means using variable calPwr
# rs_means_pwr = np.mean(rasterstack,axis=(1,2))
rs_means_pwr = np.mean(rs2,axis=(1,2))
 #then use ma.mean
# 3. Convert resulting mean value time-series to dB scale for visualization
rs_means_dB = 10.*np.log10(rs_means_pwr)
print(rs_means_pwr)

In [ ]:
# 4. Now let's plot the time series of means
plt.rcParams.update({'font.size': 14})
fig=plt.figure(figsize=(16,4))
ax1=fig.add_subplot(111)
yhat = scipy.signal.savgol_filter(rs_means_pwr, 11, 4)
ax1.plot(tindex,yhat, color='red', marker='o', markerfacecolor='white', linewidth=3, markersize=6)
ax1.plot(tindex,rs_means_pwr, color='gray', linewidth=.5)
plt.grid()
ax1.set_xlabel('Date')
ax1.set_ylabel('$\overline{\gamma^o}$ [power]')
plt.savefig('RCSoverTime.png',dpi=300,transparent='true')

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.6 Calculate Coefficient of Variance </b> </font>

<font face="Calibri" size="3"> The coefficient of variance describes how much the $\sigma_{0}$ or $\gamma_{0}$ measurements in a pixel vary over time. Hence, the coefficient of variance can indicate different vegetation cover and soil moisture regimes in your area.</font> 

In [ ]:
test = np.var(rasterstack,0)
mtest = np.mean(rasterstack[rasterstack.nonzero()],0)
coeffvar = test/(mtest+0.001)

In [ ]:
plt.rcParams.update({'font.size': 14})
fig=plt.figure(figsize=(13,10))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(coeffvar.flatten(),5)
vmax=np.percentile(coeffvar.flatten(),95)
ax.set_title('Coefficient of Variance Map')

im = ax.imshow(coeffvar,cmap='jet',vmin=vmin,vmax=vmax)
fig.colorbar(im, ax=ax)
plt.savefig('Coeffvar.png',dpi=300,transparent='true')

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.7 Threshold Coefficient of Variance Map </b> </font>

<font face="Calibri" size="3"> This is an example how to threshold the derived coefficient of variance map. This can be useful, e.g., to detect areas of active agriculture.</font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14,6)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)
# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax1.hist(coeffvar.flatten(),bins=200,range=(0,0.03))
ax1.xaxis.set_label_text('Coefficient of Variation')
ax1.set_title('Coeffvar Histogram')
plt.grid()
n, bins, patches = ax2.hist(coeffvar.flatten(), bins=200, range=(0,0.03), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Coefficient of Variation')
ax2.set_title('Coeffvar CDF')
plt.grid()

In [ ]:
plt.rcParams.update({'font.size': 14})
outind = np.where(n > 0.85)
threshind = np.min(outind)
thresh = bins[threshind]
coeffvarthresh = np.copy(coeffvar)
coeffvarthresh[coeffvarthresh < thresh] = 0
coeffvarthresh[coeffvarthresh > 0.1] = 0
fig=plt.figure(figsize=(13,10))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(coeffvar.flatten(),5)
vmax=np.percentile(coeffvar.flatten(),95)
ax.set_title(r'Thresholded Coeffvar Map [$\alpha=95%$]')

im = ax.imshow(coeffvarthresh,cmap='jet',vmin=vmin,vmax=vmax)
fig.colorbar(im, ax=ax)
plt.savefig('Coeffvarthresh.png',dpi=300,transparent='true')

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - March 2019 </i>
</font>